In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow import keras
from keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
import math
from sklearn.metrics import mean_squared_error
from IPython.display import Image

In [ ]:
df = pd.read_csv('/kaggle/input/apple-stock-data-updated-till-22jun2021/AAPL.csv')

df = pd.DataFrame(df)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
data = df.reset_index()['Close'] # Make Sure You add a space
data.head()

In [ ]:
# Using the Matplotlib Library for visualizing our time-series data
import matplotlib.pyplot as plt
plt.title("Data Plot")
plt.xlabel("Index")
plt.ylabel("Data")
plt.plot(data)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
df1 = scaler.fit_transform(np.array(data).reshape(-1,1))
df1

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df1, test_size=0.20, shuffle=False)

In [ ]:
plt.title("Data Plot")
plt.xlabel("Index")
plt.ylabel("Data")
plt.plot(X_train)

In [ ]:
plt.title("Data Plot")
plt.xlabel("Index")
plt.ylabel("Data")
plt.plot(X_test)

In [ ]:
X_train_data = []
Y_train_data = []
X_test_data = []
Y_test_data = []

train_len = len(X_train)
test_len = len(X_test)

# Create the training dataset
for i in range(train_len-101):
    a = X_train[i:(i+100), 0]
    X_train_data.append(a)
    Y_train_data.append(X_train[i + 100, 0])

# Create the test dataset
for j in range(test_len-101):
    b = X_test[j:(j+100), 0]
    X_test_data.append(b)
    Y_test_data.append(X_test[j + 100, 0])

X_train_data = np.array(X_train_data)
Y_train_data = np.array(Y_train_data)
X_test_data = np.array(X_test_data)
Y_test_data = np.array(Y_test_data)

In [ ]:
X_train_data

In [ ]:
### Printing the training and testing shapes

print("Training size of data = ", X_train_data.shape)
print("Training size of labels = ", Y_train_data.shape)
print("Training size of data = ", X_test_data.shape)
print("Training size of labels = ", Y_test_data.shape)

In [ ]:
### Converting the training and testing data shapes into a 3-dimensional space to make it suitable for LSTMs

X_train_data_3d = X_train_data.reshape(X_train_data.shape[0], X_train_data.shape[1], 1)
X_test_data_3d = X_test_data.reshape(X_test_data.shape[0], X_test_data.shape[1], 1)

print(X_train_data_3d.shape)
print(X_test_data_3d.shape)

In [ ]:
#LSTM model

model=Sequential()
model.add(LSTM(units = 100,return_sequences = True,input_shape = (X_train_data_3d.shape[1],1)))
#model.add(Dropout(0.1))
model.add(LSTM(units = 100)) # Removed return_sequences = True
#model.add(Dropout(0.1))
model.add(Dense(units = 1)) # Removed extra Dense layer

model.summary()

In [ ]:
keras.utils.plot_model(model, to_file='model.png', show_layer_names=True)

In [ ]:
checkpoint = ModelCheckpoint("checkpoint1.h5", monitor='val_loss', verbose=1,
                              save_best_only=True, mode='auto')

logdir='logs1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [ ]:
# Model Compilation

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Training The Model

model.fit(X_train_data,
          Y_train_data,
          validation_data=(X_test_data, Y_test_data),
          epochs=50,
          batch_size=128,
          verbose=1,
          callbacks=[checkpoint, tensorboard_Visualization])

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(model.history.history['loss'], label='Training Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_predict = model.predict(X_train_data)
test_predict = model.predict(X_test_data)

# Transform back to original form
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

In [ ]:
print("Train MSE = ", math.sqrt(mean_squared_error(Y_train_data, train_predict)))
print("Test MSE = ", math.sqrt(mean_squared_error(Y_test_data, test_predict)))

In [ ]:
# Plotting
# Shift train predictions for plotting
look_back = 100
data_array = data.values.reshape(-1, 1)

# Inverse transform predictions to original scale
train_predict_plot = scaler.inverse_transform(train_predict)
test_predict_plot = scaler.inverse_transform(test_predict)

trainPredictPlot = np.empty_like(data_array)
trainPredictPlot[:] = np.nan
trainPredictPlot[look_back:len(train_predict_plot)+look_back] = train_predict_plot

# Shift test predictions for plotting
testPredictPlot = np.empty_like(data_array)
testPredictPlot[:] = np.nan
testPredictPlot[len(train_predict_plot)+(look_back*2)+1:len(data_array)-1] = test_predict_plot

# Plot baseline and predictions
plt.figure(figsize=(12, 6))
plt.plot(scaler.inverse_transform(data_array), label='Original Data')
plt.plot(trainPredictPlot, label='Train Predictions')
plt.plot(testPredictPlot, label='Test Predictions')
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

# Calculate MAPE for train and test predictions
train_mape = mean_absolute_percentage_error(scaler.inverse_transform(Y_train_data.reshape(-1, 1)), train_predict)
test_mape = mean_absolute_percentage_error(scaler.inverse_transform(Y_test_data.reshape(-1, 1)), test_predict)

# Get RMSE values from previous output
train_rmse = math.sqrt(mean_squared_error(scaler.inverse_transform(Y_train_data.reshape(-1, 1)), train_predict))
test_rmse = math.sqrt(mean_squared_error(scaler.inverse_transform(Y_test_data.reshape(-1, 1)), test_predict))

# Create a comparison table
comparison_data = {'Metric': ['RMSE', 'MAPE'],
                   'Train': [train_rmse, train_mape],
                   'Test': [test_rmse, test_mape]}
comparison_df = pd.DataFrame(comparison_data)

display(comparison_df)